<h2 style="color:#22198A">PROJECT INFO</h2>

<h3 style="color:green">About project-01</h3>
<p>This project is focused analysing data from UK government concerning accidents in the UK in year 2019. Our group specifically analysed <b>Manchester.</b></p>
<p><b>Contact:</b> luci@itu.dk, mdon@itu.dk, jses@itu.dk, mksi@itu.dk</p>
<p><b>Created:</b> 08. 02. 2021</p>
<p><b>Last modified:</b> 25. 02. 2021</p>

<h2 style="color:#22198A">NOTEBOOK SETUP</h2>
<p>Before you start working with notebook, please make sure to go through this setup to ensure smooth running. (by default, no changes should be needed if you just downloaded the repository)</p>
<h3 style="color:green">Important highlights</h3>
<ul>
<li><b>BASE_DIR:</b> This should lead to the root directory relative to location of this notebook</li>
<li><b>SCRIPTS IMPORT:</b> All scripts are saved within one file. In the file, there are comments splitting the whole file into sections which gather scripts with similar functionality, e.g. loading data. All functions should contain docstring, which might be useful for any troubleshooting or just knowing how the given thing was implemented. The way the scripts are imported was implemented according to <a href='# https://stackoverflow.com/questions/34478398/import-local-function-from-a-module-housed-in-another-directory-with-relative-im
'>this</a> SO question. <b>Once you run the below cell, all scripts should be loaded.</b></li>
<li><b>PACKAGES USED WITHIN DIRECTORY: </b> Most of the functionality is written in <b>all_scripts.py</b> where you can see in the beginning all the packages used, but it is worth highlight these "not so standard" packages which you should make sure you have installed: <b>pandas, scipy, branca, geopy, sklearn, openpyxl</b></li>
</ul>

In [ ]:
BASE_DIR = ""

# SCRIPTS IMPORT
import os
import sys
scripts_path = os.path.abspath(os.path.join(f'{BASE_DIR}scripts'))

if scripts_path not in sys.path:
    # Add the scripts to the path
    sys.path.append(scripts_path)
    
    # Import the needed scripts
    from all_scripts import *
    
    # Remove the added path to avoid possible future conflicts
    sys.path.remove(scripts_path)
else:
    
    # Import the needed scripts
    from all_scripts import *
    
    # Remove the added path to avoid possible future conflicts
    sys.path.remove(scripts_path)

<h2 style="color:#22198A">CONSTANTS</h2>
<h3 style="color:green">Important highlights</h3>
<ul>
<li><b>PATH_REFERENCES:</b> There was an original variable lookup table  which we modified, so var lookup leads to this modified table. The reason for this step was being able to use column names from individual datasets to index Sheet names in the variable lookup file. The problem was that certain sheet names had unfortunately different names than column names. For more detail, look at script <b>map_num_cat_to_named_cat</b></li>
<li><b>NAMING CONVENTIONS - TABLES:</b> Since there are three datasets, we usually store these throughout the notebook using a dictionary. The important thing to notice, that the dictionary always has the following keys referencing to tables: <b>accidents, casualties, vehicles</b></li>
</ul>

In [ ]:
PATH_DATA = {
    "raw": f"{BASE_DIR}data/raw/",
    "interim": f"{BASE_DIR}data/interim/",
    "external": f"{BASE_DIR}data/external/"
    
}

PATH_REFERENCES = {
    "var_lookup": f"{BASE_DIR}references/variable_lookup_modified.xlsx"
}

FILENAME_DATA = {
    "accidents": "Road Safety Data - Accidents 2019.csv",
    "casualties": "Road Safety Data - Casualties 2019.csv",
    "vehicles": "Road Safety Data- Vehicles 2019.csv",
}

<h2 style="color:#22198A">LOAD RAW DATA</h2>
<h3 style="color:green">Important highlights</h3>
<ul>
<li><b>LOADING RAW DATA:</b> We loaded the data as a masked numpy array. The only difference compare to lecture is that we also used parameter <b>missing_values</b> and set it to {'-1', ''} which specifies explicitly how are defined missing values. We made the decision after exploring variable lookup dataset.</li>
<li><b>LOADING VAR LOOKUP</b> Since we modified the file, the format of excel sheet is not xls anymore, but xlsx (more modern version of storing excel data <a href='https://stackoverflow.com/questions/65250207/pandas-cannot-open-an-excel-xlsx-file'>since 2007</a>), you might need to install <b>openpyxl</b>, if you are using Anacando, then read how <a href=https://anaconda.org/anaconda/openpyxl>here</a>. For troubleshooting, see this <a href=https://stackoverflow.com/questions/65250207/pandas-cannot-open-an-excel-xlsx-file>SO answer.</a></li>
</ul>

In [ ]:
RAW_DATA = {
    key: load_data_numpy_csv(PATH_DATA['raw'] + filename)
    for key, filename in FILENAME_DATA.items()
}

VAR_LOOKUP = load_pandas_df_from_excel(PATH_REFERENCES['var_lookup'])

In [ ]:
for table_name, records in RAW_DATA.items():
    print(f'There is in total {records.shape[0]} records in {table_name} table.')

<h2 style="color:#22198A">TASK 0: DATA FILTERING, CLEANING AND EXPLORING</h2>
<h3 style="color:green">Get only accidents related to Manchester</h3>
<br>
<p> We decided to use Local Authority District code to filter out the relvant results for Manchester. See the process below.</p>
<p><b>NOTE:</b> We only considered Manchester as a city and not the whole Greater Manchester area. The reason is that Manchester as a city has its own local government and even more importantly we were tasked to analyse Manchester as a city.</p>

<h4 style="color:#ff9900">EXPLORE POSIBILITIES FOR SELECTING MANCHESTER CITY DATA.</h4>
<p>After the exploration of variables within accident table we found two variables which should be relevant: <b>Local_Authority_Highway and Local_Authority_District</b></p>

In [ ]:
# Check validity of the values
missing_vals_highway = np.count_nonzero(RAW_DATA['accidents']['Local_Authority_Highway'].mask)
missing_vals_district = np.count_nonzero(RAW_DATA['accidents']['Local_Authority_District'].mask)
print(f"Local authority district has {missing_vals_district} missing values and Local Authority Highway has {missing_vals_highway} missing values.")

<p>This seems to be a good idea. In fact there is a wikipedia webpage showing that Local authority district is something which we should look for since there are different districts within the Greater Manchester city area and Manchester city is one of them with its own local council. <a href="https://en.wikipedia.org/wiki/Category:Local_authorities_in_Greater_Manchester">See it here.</a></p>

In [ ]:
# Filter the Manchester records based on the two variables and see if they match
manchester_only_mask_highway = RAW_DATA['accidents']['Local_Authority_Highway'] == 'E08000003'
manchester_only_highway = RAW_DATA['accidents'][manchester_only_mask_highway]

manchester_only_mask_district = RAW_DATA['accidents']['Local_Authority_District'] == 102
manchester_only_district = RAW_DATA['accidents'][manchester_only_mask_district]
print(f"Using the Highway code, we obtained {manchester_only_highway.shape[0]} records and the district code we obtained {manchester_only_district.shape[0]} records.")

<h4 style="color:#ff9900">SELECT THE DATA FOR MANCHESTER</h4>
<p>Now, that we know the two variables yielded the same result, we decided to choose Local Authority District as the one to rely on during the rest of the analysis.</p>

In [ ]:
# Get accidents relevant to Manchester
ACCIDENTS_IN_MANCHESTER = get_city_specific_records_by_lad(RAW_DATA['accidents'], 102)
print(f'There is {ACCIDENTS_IN_MANCHESTER.shape[0]} accident records related to MANCHESTER.')

<h3 style="color:green">Use accident indexes related to Manchester to select relevant data from other two tables</h3>
<h4 style="color:#ff9900">SANITY CHECK: DOES EVERY ACCIDENT ID IN THE CASUALTIES AND VEHICLES TABLE HAVE THEIR CORESSPONDING ACCIDENT ID IN THE ACCIDENT TABLE?</h4>

In [ ]:
for other_table in RAW_DATA.keys():
    if other_table != 'accidents':
        print(f'Table: {other_table.upper()}')
        non_existent_ids = is_accident_id_existent(RAW_DATA['accidents'], RAW_DATA[other_table])
        if not non_existent_ids:
            print(f'- OK: All Accident IDs in {other_table} have corresponding ID in accident table.')
        else:
            print(non_existent_ids)
        print()

<h4 style="color:#ff9900">SELECT RELEVANT RECORDS FOR MANCHESTER FROM OTHER TWO TABLES</h4>

In [ ]:
# Mask
mask_casualties = np.isin(RAW_DATA['casualties']['Accident_Index'], ACCIDENTS_IN_MANCHESTER['Accident_Index'])
mask_vehicles = np.isin(RAW_DATA['vehicles']['Accident_Index'], ACCIDENTS_IN_MANCHESTER['Accident_Index'])

# Select the corresponding records
CASUALTIES_IN_MANCHESTER = RAW_DATA['casualties'][mask_casualties]
VEHICLES_IN_MANCHESTER = RAW_DATA['vehicles'][mask_vehicles]

print(f'There are {CASUALTIES_IN_MANCHESTER.shape[0]} casuality records and {VEHICLES_IN_MANCHESTER.shape[0]} vehicles records.', )

<h3 style="color:green">Select only relevant columns from all tables</h3>

In [ ]:
# TODO: Add description why we chose given columns
MANCHESTER_ONLY_RECORDS = {
    'accidents': ACCIDENTS_IN_MANCHESTER,
    'casualties': CASUALTIES_IN_MANCHESTER,
    'vehicles': VEHICLES_IN_MANCHESTER
}


SELECTED_COLUMNS = {
    'accidents': [
        'Accident_Index',
        'Longitude',
        'Latitude',
        'Accident_Severity',
        'Number_of_Vehicles',
        'Number_of_Casualties',
        'Date',
        'Day_of_Week',
        'Road_Type',
        'Time',
        'Speed_limit',
        'Junction_Detail',
        'Junction_Control',
        'Pedestrian_CrossingHuman_Control',
        'Light_Conditions',
        'Weather_Conditions',
        'Road_Surface_Conditions',
        'Special_Conditions_at_Site',
        'Carriageway_Hazards'
    ],
    'casualties': [
        'Accident_Index',
        'Casualty_Class',
        'Sex_of_Casualty',
        'Age_of_Casualty',
        'Age_Band_of_Casualty',
        'Pedestrian_Location',
        'Pedestrian_Movement',
        'Car_Passenger',
        'Casualty_Severity',
        'Bus_or_Coach_Passenger',
        'Casualty_Type'
    ],
    'vehicles': [
        'Accident_Index',
        'Towing_and_Articulation',
        'Vehicle_Manoeuvre',
        'Vehicle_LocationRestricted_Lane',
        'Junction_Location',
        'Skidding_and_Overturning',
        'Hit_Object_in_Carriageway',
        'Vehicle_Leaving_Carriageway',
        'Hit_Object_off_Carriageway',
        '1st_Point_of_Impact',
        'Sex_of_Driver',
        'Age_of_Driver',
        'Age_Band_of_Driver',
        'Age_of_Vehicle'
    ]
}

SELECTED_DATA = {
    key: select_only_relevant_columns(MANCHESTER_ONLY_RECORDS[key], sel_columns)
    for key, sel_columns in SELECTED_COLUMNS.items()
}

<h3 style="color:green">Do a summary of the adjusted tables</h3>
<h4 style="color:#ff9900">INFORMATION ABOUT THE FILTERED DATASETS</h4>

In [ ]:
datasets_info =  get_info_datasets(SELECTED_DATA)
for key, info in datasets_info.items():
    print(f'{key}'.upper())
    for info_key, value in info.items():
        if info_key != "column names":
            print(info_key, ': ', value)
    print()

<h4 style="color:#ff9900"> NUMERICAL DATA: FIVE NUMBER SUMMARY</h4>
<p>Select below, whether you want to see 5 number summary graphically (boxplots) or not.</p>

In [ ]:
PLOT_BOXPLOTS = False

<p>Define columns with numerical data for each table.</p>

In [ ]:
NUMERICAL_DATA = {
    'accidents': ['Number_of_Vehicles', 'Number_of_Casualties', 'Speed_limit'],
    'vehicles': ['Age_of_Driver', 'Age_of_Vehicle'],
    'casualties': ['Age_of_Casualty']
}

In [ ]:
# Check that there are no missing values in selected columns
for table, column_names in NUMERICAL_DATA.items():
    print(f"In the table {table}")
    for column_name in column_names:
        print(f"  there is in total {np.count_nonzero(SELECTED_DATA[table][column_name].mask)} missing values in the column {column_name}")

In [ ]:
# Visual 5 number summary
if PLOT_BOXPLOTS:
    for table_name, num_col_names in NUMERICAL_DATA.items():
        boxplots(SELECTED_DATA[table_name], num_col_names, n_rows=1)

# 5 number summary for numerical data
else:
    print('==========================================')
    print()
    for table_name, num_col_names in NUMERICAL_DATA.items():    
        five_num_summary = get_5_num_sum(SELECTED_DATA[table_name], columns=num_col_names)
        print(f'{table_name.upper()} TABLE')
        for column_name, values in five_num_summary.items():
            print(f'- {column_name.upper()}')
            for key, number in values.items():
                print(f'  {key}: {number}')
            print()
        print()
        print('==========================================')
        print()

<h4 style="color:#ff9900"> CATEGORICAL DATA - ACCIDENTS TABLE</h4>

In [ ]:
selected_column_names_accidents = [
    'Accident_Severity',
    'Day_of_Week',
    'Road_Type',
    'Speed_limit',
    'Junction_Detail',
    'Junction_Control',
    'Light_Conditions',
    'Weather_Conditions',
    'Road_Surface_Conditions',
    'Special_Conditions_at_Site',
    'Carriageway_Hazards',
    'Pedestrian_CrossingHuman_Control'
    ]
freqBar(SELECTED_DATA['accidents'],
        VAR_LOOKUP,
        selected_column_names_accidents,
        figheight=30,
        figwidth=15,
        hspace=0.8,
        nrows=4,
        check_missing_vals=True)

<h4 style="color:#ff9900"> CATEGORICAL DATA - VEHICLES TABLE</h4>

In [ ]:
selected_column_names_vehicles = [
    'Towing_and_Articulation',
    'Vehicle_Manoeuvre',
    'Junction_Location',
    'Skidding_and_Overturning',
    'Hit_Object_in_Carriageway',
    'Vehicle_Leaving_Carriageway',
    'Hit_Object_off_Carriageway',
    '1st_Point_of_Impact',
    'Sex_of_Driver'
    ]
freqBar(SELECTED_DATA['vehicles'],
        VAR_LOOKUP,
        selected_column_names_vehicles,
        figheight=35,
        figwidth=15,
        hspace=0.8,
        check_missing_vals=True)

<h4 style="color:#ff9900"> CATEGORICAL DATA - CASUALTIES TABLE</h4>

In [ ]:
selected_column_names_casualties = [
    'Casualty_Class',
    'Sex_of_Casualty',
    'Casualty_Severity',
    'Pedestrian_Location',
    'Car_Passenger',
    'Bus_or_Coach_Passenger',
    'Casualty_Type'
]
freqBar(SELECTED_DATA['casualties'],
        VAR_LOOKUP,
        selected_column_names_casualties,
        figheight=35,
        figwidth=17,
        hspace=0.5,
        nrows=3,
        check_missing_vals=True)

<h3 style="color:green">Save the adjusted tables to data/interim</h3>

In [ ]:
for key, arr in SELECTED_DATA.items():
    
    # Get proper filename
    fname = FILENAME_DATA[key]
    
    # Get filepath
    filepath = f"{PATH_DATA['interim']}{fname}"
    header = ','.join(arr.dtype.names)
    np.savetxt(filepath, arr, fmt='%s', delimiter=',', header=header)

<h2 style="color:#22198A">Task 1: Single variable analysis</h2>

In [ ]:
CLEAN_DATA = {
    key: load_data_numpy_csv(PATH_DATA['interim'] + filename)
    for key, filename in FILENAME_DATA.items()
}

<h3 style="color:green">Age of driver</h3>

In [ ]:
# Get valid data since we know there is 234 records missing
no_missing_vals_mask = ~(CLEAN_DATA['vehicles']['Age_of_Driver'].mask)
age_driver_defined_vals_only = CLEAN_DATA['vehicles']['Age_of_Driver'][no_missing_vals_mask]

# Plot the graph
histograms(CLEAN_DATA['vehicles']['Age_of_Driver'],
           selected_column_names=['Age of Driver'],
           figheight=5,
           figwidth=15,
           hspace=0.5,
           nrows=1,
           nbins=25,
          xticks=[i for i in range(0, 110, 10)])

<h3 style="color:green">Histogram of time, day and date</h3>
<h4 style="color:#ff9900">TIME</h4>

In [ ]:
# Get all time data which are in the format hh:mm
time_data = CLEAN_DATA['accidents']['Time']

# Parse hours from the data
hours = [strip_number_from_zero(time[0]) for time in np.char.split(time_data, sep=":")]

# Convert the list into array and use int as dtype
hours_arr = np.array(hours).astype(int)

# Plot the data
histograms(hours_arr,
           selected_column_names=['Accident hours'],
           figheight=5,
           figwidth=17,
           hspace=0.5,
           nrows=1,
           nbins=12,
           xticks=[i for i in range(0, 26, 2)],
           range=(0, 24)
          )

<h4 style="color:#ff9900">WEEKDAY</h4>

In [ ]:
weekday_data = CLEAN_DATA['accidents']['Day_of_Week']

freqBar(weekday_data,
        VAR_LOOKUP,
        selected_column_names=['Day_of_Week'],
        figheight=5,
        figwidth=15,
        hspace=0.5,
        nrows=1)

<h4 style="color:#ff9900">MONTH TOTAL</h4>

In [ ]:
# Get date data
date_data = CLEAN_DATA['accidents']['Date']

# Parse months data from it
months = [strip_number_from_zero(elements[1]) for elements in np.char.split(date_data, sep="/")]

# Convert the list into array and use int as dtype
months_arr = np.array(months).astype(int)

# Create xlabels
month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun', 'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

# Plot the data
freqBar(months_arr,
        VAR_LOOKUP,
        selected_column_names=['Accident months'],
        figheight=5,
        figwidth=15,
        hspace=0.5,
        nrows=1,
        xlabels=month_names
       )

<h4 style="color:#ff9900">MONTH SPLIT INTO DAYS</h4>

In [ ]:
# Parse data into a dictionary
summary_dic = dict()
for date in date_data:
    
    # Parse date
    day, month, _ = date.split('/')
    
    # Modify the inputs if needed
    day, month = strip_number_from_zero(day), strip_number_from_zero(month)
    
    # Convert month number to month name
    month_name = month_names[int(month)-1]
    
    # Save it to a dictionary
    if month_name in summary_dic:
        summary_dic[month_name].append(int(day))
    else:
        summary_dic[month_name] = [int(day)]

# Plot the data
histograms(summary_dic,
           selected_column_names=[key for key in summary_dic],
           figheight=15,
           figwidth=17,
           hspace=0.9,
           nrows=4,
           nbins=15,
           xticks=[i for i in range(1, 31, 2)],
           range=(1, 31)
          )

<h3 style="color:green">Casualty Types, Road Types and Severity</h3>

<h4 style="color:#ff9900">FREQUENCY OF ACCIDENTS BY SEVERITY</h4>

In [ ]:
freqBar(CLEAN_DATA['casualties']['Casualty_Severity'],
        VAR_LOOKUP,
        selected_column_names=['Casualty_Severity'],
        figheight=5,
        figwidth=25,
        nrows=1)

In [ ]:
map_sev = {
    1: 'Fatal',
    2: 'Serious',
    3: 'Slight'
}
for sev, sev_name in map_sev.items():
    mask_cas_severity = CLEAN_DATA['casualties']['Casualty_Severity'] == sev
    number_of_impacted = CLEAN_DATA['casualties']['Casualty_Severity'][mask_cas_severity].shape[0]
    print(f'There were {number_of_impacted} {sev_name} casualties in Manchester in 2019.')

<h4 style="color:#ff9900">FREQUENCY OF ACCIDENTS BY CASUALTY TYPE</h4>

In [ ]:
casualty_type_data = CLEAN_DATA['casualties']['Casualty_Type']

freqBar(casualty_type_data,
        VAR_LOOKUP,
        selected_column_names=['Casualty_Type'],
        figheight=4,
        figwidth=15,
        nrows=1)

<h4 style="color:#ff9900">CLOSER LOOK AT SELECTED CASULTY TYPES</h4>

In [ ]:
# Create summary dictionary for casuality data
summary_casuality = dict()

# Add casualties selected by severity and filtered by major casualty types
column_names = ['Fatal severity', 'Serious severity', 'Slight severity']
for severity_name, severity in zip(column_names, [1, 2, 3]):
    
    # Select casualties with the given severity
    severity_mask = CLEAN_DATA['casualties']['Casualty_Severity'] == severity
    
    # Select only certain Casualty types
    casuality_type_mask = np.isin(CLEAN_DATA['casualties']['Casualty_Type'], np.array([0, 1, 9]))
    
    # Select given records using the mask
    selected_casualties = CLEAN_DATA['casualties'][(severity_mask) & (casuality_type_mask)]
    
    # Add these data to summary dictionary
    summary_casuality[severity_name] = selected_casualties['Casualty_Type']

freqBar(summary_casuality,
        VAR_LOOKUP,
        selected_column_names=column_names,
        figheight=4,
        figwidth=15,
        nrows=1,
        xlabels=['Pedestrian', 'Cyclist', 'Car occupant']
       )

<h4 style="color:#ff9900">ROAD TYPES</h4>

In [ ]:
mask_cas_severity = CLEAN_DATA['casualties']['Casualty_Severity'] == 3
selected_cas_accident_ids = CLEAN_DATA['casualties']['Accident_Index'][mask_cas_severity]
mask_relevant_accidents = np.isin(CLEAN_DATA['accidents']['Accident_Index'], selected_cas_accident_ids)
road_types = CLEAN_DATA['accidents']['Road_Type'][mask_relevant_accidents]

freqBar(road_types,
        VAR_LOOKUP,
        selected_column_names=['Road_Type'],
        figheight=4,
        figwidth=15,
        nrows=1)

<h2 style="color:#22198A">Task 2: Associations</h2>
<h3 style="color:green">Junction detail versus Casualty severity</h3>
<ul>
<li>$H_0$ (null hypothesis): There is no statistically significant relationship between junction detail and casualty severity.</li>
<li>$H_\alpha$ (alternative hypothesis): There is statistically significant relationship between junction detail and casualty severity.</li>
</ul>
<p>If the p-value $< \alpha$, then the null hypothesis will be rejected.</p>
<h4 style="color:#ff9900">PREPARE THE TEST DATA</h4>
<ul>
<li><b>NOTE:</b> We had to adjust junction detail records since in crosstable we got many zeroes, therefore we decided to narrow the junction detail into the following categories: <b>Crossroads, Not at junction or within 20 metres, T or stagerred junction</b></li>
</ul>

In [ ]:
# Load the data
junction_detail = CLEAN_DATA['accidents']["Junction_Detail"]
accidents_index = CLEAN_DATA['accidents']["Accident_Index"]
casualty_severity = CLEAN_DATA['casualties']["Casualty_Severity"]
casualty_acc_index = CLEAN_DATA['casualties']["Accident_Index"]

# Narrow down the categories
# Accidents table
selected_junction_categories = [0,3, 6]
selected_cat_junction_mask = np.isin(junction_detail,
                                     np.array(selected_junction_categories))
junction_detail = junction_detail[selected_cat_junction_mask]
accidents_index = accidents_index[selected_cat_junction_mask]
# Casualties table
acc_mask = np.isin(casualty_acc_index, accidents_index)
casualty_severity = casualty_severity[acc_mask]
casualty_acc_index = casualty_acc_index[acc_mask]

# Since there is more casualties than accidents,
# we need to edit junction detail column so its number of rows corresponds
# to number of rows in casualty severity
new_junction_detail = np.array([junction_detail[accidents_index == cas_index].data[0]
                                for cas_index in casualty_acc_index])

# Map column values representing codes to its correspnding
# names using the provided lookup table
junction_detail_named = map_num_cat_to_named_cat(VAR_LOOKUP,
                                                 new_junction_detail,
                                                 'Junction_Detail')

casualty_severity_named = map_num_cat_to_named_cat(VAR_LOOKUP,
                                                   casualty_severity,
                                                   'Casualty_Severity')

<h4 style="color:#ff9900">CONDUCT THE PEARSON CHI-SQUARE TEST</h4>

In [ ]:
observed_pd, expected_pd, chiVal, pVal, df = do_pearson_chi_square_test(
    junction_detail_named,
    casualty_severity_named,
    ['Junction detail'],
    ['Casualty Severity']
)
chiVal

In [ ]:
observed_pd

In [ ]:
expected_pd

In [ ]:
print(f'pValue is {round(pVal, 3)}')

<h4 style="color:#ff9900">COMPUTE CRAMER'S V</h4>

In [ ]:
cramers_v = get_cramers_v(chiVal, observed_pd)
print(f'Cramers V is {round(cramers_v, 2)} which means a weak relationship.')

<h4 style="color:#ff9900">PREPARE THE DATA FOR VISUAL REPORT</h4>

In [ ]:
# Get category names for both categorical variables
# Casualty Severity
df_severity = VAR_LOOKUP['Casualty Severity']
cats_severity = np.unique(casualty_severity)
cats_severity_named = get_relevant_lookup_values(df_severity, cats_severity)
# Junction detail
df_junction = VAR_LOOKUP['Junction Detail']
cats_junction = [6,0, 3]
cats_junction_named =  get_relevant_lookup_values(df_junction, cats_junction)
# Define x ticks
xticks = [i for i in range(1, len(cats_junction)+1)]

<h4 style="color:#ff9900">MAKE A VISUAL REPORT</h4>

In [ ]:
# Create a figure and axes object
fig, axs = plt.subplots(nrows=2, ncols=3, figsize=(12, 6))

# Setup spaces horizontally and vertically
fig.subplots_adjust(left=None, bottom=None, right=None,
                        top=None, wspace=0.2, hspace=0.3)

# Iterate over the first row
row1 = axs[0]
index = 0
for ax in row1:
    
    # Plot the values
    ax.plot(xticks, list(observed_pd.iloc[:, index]), 'ro-', label='Observed')
    ax.plot(xticks, list(expected_pd.iloc[:, index]), 'bo-', label='Expected')
    
    # Plot the legend within the first plot
    if index == 0: 
        ax.set_ylabel('Casualties')
        ax.legend(loc='best');
        
    # Name the axes, set the title
    ax.set_title(f"{cats_severity_named[index]}\n", fontweight="bold", fontsize=14)
    ax.set_xticks(xticks)
    ax.set_xticklabels(cats_junction_named)
    fig.autofmt_xdate(rotation=20)
    
    index += 1

row2 = axs[1]
index = 0
for ax in row2:
    
    # Plot the values
    ax.plot(xticks, observed_pd.iloc[:, index]/expected_pd.iloc[:, index], 'go-')
    ax.plot(xticks, [1 for _ in range(len(xticks))], 'k:')
    
    # Name the axes, set the title
    if index == 0: 
        ax.set_ylabel('Observed/Expected')
    ax.set_xticks(xticks)
    ax.set_xticklabels(cats_junction_named)
    fig.autofmt_xdate(rotation=20)
    
    index += 1
# fig.savefig('task_2.png', dpi=300, facecolor='w', edgecolor='w',
#                     orientation='portrait')

<h2 style="color:#22198A">Task 3: Map visualization</h2>
<h3 style="color:green">Casualty visualiazation on map</h3>
<h4 style="color:#ff9900">GET RELEVANT DATA</h4>

In [ ]:
relevant_data = {
    'Casualty_Severity': CLEAN_DATA['casualties']['Casualty_Severity'],
    'Latitude': [],
    'Longitude': []
}

for accident_index_casualty in CLEAN_DATA['casualties']['Accident_Index']:
    
    # Get longitude and latitude from the accident table
    index_mask = np.isin(CLEAN_DATA['accidents']['Accident_Index'], accident_index_casualty)
    relevant_data['Longitude'].append(CLEAN_DATA['accidents'][index_mask]['Longitude'].data[0])
    relevant_data['Latitude'].append(CLEAN_DATA['accidents'][index_mask]['Latitude'].data[0])

vals = [(cas, lat, lon) for cas, lat, lon
        in zip(relevant_data['Casualty_Severity'],
               relevant_data['Latitude'],
               relevant_data['Longitude'])]

relevant_data = np.array(vals,
                         dtype=[('Casualty_Severity', int),
                                ('Latitude', float),
                                ('Longitude', float)])

<h4 style="color:#ff9900">PLOT IT WITH GIVEN SETTINGS</h4>

In [ ]:
# Create color map
color_map = {
    1: 'red',
    2: 'orange',
    3: 'blue'
}

# Get folium map
M = create_folium_map(relevant_data, color_map=color_map, column_to_highlight='Casualty_Severity')

# Add legend
color_map_named = {
    'Fatal': 'red',
    'Serious': 'orange',
    'Slight': 'blue'
}
add_legend(M, color_map_named)

<h2 style="color:#22198A">Task 4: Open question</h2>
<p><b>What are the hotspots for accidents over several years and how can better safety can be assured at these places?</b></p>
<h3 style="color:green">Load data needed for task 4</h3>
<p><b>Note1:</b> Might take a bit longer since you are loading a big amount of data.</p>
<p><b>Note2:</b> Accident data from other years are loaded exactly same way as accident data from year 2019, therefore we do no provide any further clarification on the way we clean them or process since this was done in previous section.</p>
<p><b>Note3:</b> Data were obtained at this <a href="https://data.gov.uk/dataset/cb7ae6f0-4be6-4935-9277-47e5ce24a11f/road-safety-data">official website</a>, which should be identical to the source of data for year 2019.</p>

In [ ]:
FILENAME = 'dftRoadSafetyData_Accidents_'
YEARS = ['2015', '2016', '2017', '2018']
OTHER_YEARS_ACCIDENTS = {
    year: get_accident_data_from_given_year(f"{PATH_DATA['external']}{FILENAME}{year}.csv")
    for year in YEARS
}
ALL_YEARS_ACCIDENTS = {
    year: get_accident_data_from_given_year(f"{PATH_DATA['external']}{FILENAME}{year}.csv")
    for year in YEARS
}
ALL_YEARS_ACCIDENTS['2019'] = CLEAN_DATA['accidents']

<h3 style="color:green">Summary of accidents in Manchester from 2015 - 2019</h3>
<p>This section briefly summarizes the accidents statistics in Manchester from 2015 - 2019.</p>

<h4 style="color:#ff9900">NUMBER OF ACCIDENTS PER YEAR</h4>

In [ ]:
# Create a summary dictionary which will hold the values
summary_dict = dict()
summary_dict['Year of Accident'] = [data.shape[0] for year, data in OTHER_YEARS_ACCIDENTS.items()]
summary_dict['Year of Accident'].append(CLEAN_DATA['accidents'].shape[0])

# Plot the data
freqBar(summary_dict,
        VAR_LOOKUP,
        selected_column_names=['Year of Accident'],
        figheight=4,
        figwidth=15,
        hspace=0.5,
        nrows=1,
        xlabels=list(OTHER_YEARS_ACCIDENTS.keys())+ ['2019'],
        use_spec_count=True)

<h4 style="color:#ff9900">ACCIDENT SEVERITY OVER THE YEARS</h4>

In [ ]:
# Create a summary dictionary containing a year and number of accidents by severity
summary_dict = dict()

# Fatal
summary_dict['Fatal Accidents'] = [data[data['Accident_Severity'] == 1].shape[0] for year, data in OTHER_YEARS_ACCIDENTS.items()]
summary_dict['Fatal Accidents'].append(CLEAN_DATA['accidents'][CLEAN_DATA['accidents']['Accident_Severity'] == 1].shape[0])

# Serious
summary_dict['Serious Accidents'] = [data[data['Accident_Severity'] == 2].shape[0] for year, data in OTHER_YEARS_ACCIDENTS.items()]
summary_dict['Serious Accidents'].append(CLEAN_DATA['accidents'][CLEAN_DATA['accidents']['Accident_Severity'] == 2].shape[0])

# Slight
summary_dict['Slight Accidents'] = [data[data['Accident_Severity'] == 3].shape[0] for year, data in OTHER_YEARS_ACCIDENTS.items()]
summary_dict['Slight Accidents'].append(CLEAN_DATA['accidents'][CLEAN_DATA['accidents']['Accident_Severity'] == 3].shape[0])

# Plot the data
freqBar(summary_dict,
        VAR_LOOKUP,
        selected_column_names=['Fatal Accidents', 'Serious Accidents', 'Slight Accidents'],
        figheight=4,
        figwidth=15,
        hspace=0.5,
        nrows=1,
        xlabels=list(OTHER_YEARS_ACCIDENTS.keys())+ ['2019'],
        use_spec_count=True)

<h3 style="color:green">Hotsposts of accidents in Manchester - overview per year</h3>
<p>The goal of this section is to explore hotspots of accidents from previous years. Next section will then combine the clusters from different years and put them on the map to see the overall trends.</p>

<h4 style="color:#ff9900">HYPER PARAMETER SETTINGS</h4>
<ul>
<li><b>MAX DISTANCE: </b>Maximum distance between two points on the map. For more detail, see the doc string of the function.</li>
<li><b>MIN CLUSTER SIZE: </b>Minimum number of records to for a cluster to be considered.</li>
</ul>

In [ ]:
MAX_DISTANCE = 75
MIN_CLUSTER_SIZE = 3

<h4 style="color:#ff9900">ACCIDENT CLUSTERS IN 2019</h4>

In [ ]:
M_2019, meta_2019 = create_folium_map_clusters(CLEAN_DATA['accidents'],
                                               max_distance=MAX_DISTANCE,
                                               min_cluster_size=MIN_CLUSTER_SIZE,
                                               radius = 10)
M_2019

<h4 style="color:#ff9900">ACCIDENT CLUSTERS IN 2018</h4>

In [ ]:
M_2018, meta_2018 = create_folium_map_clusters(OTHER_YEARS_ACCIDENTS['2018'],
                                               max_distance=MAX_DISTANCE,
                                               min_cluster_size=MIN_CLUSTER_SIZE,
                                               radius = 10)
M_2018

<h4 style="color:#ff9900">ACCIDENT CLUSTERS IN 2017</h4>

In [ ]:
# Hack around getting 2017 work ––> for some reason latitude and longitude data
# were loaded as strings and not as floats as in other instances which caused fill value to be
# unacceptable when converting strings to floats. The problem got solved by explicitly changing the fill value.
# NOTE: Since there were not any missing values, it should not any how affect the functionality.
OTHER_YEARS_ACCIDENTS['2017']['Latitude'].fill_value = 10**9
OTHER_YEARS_ACCIDENTS['2017']['Longitude'].fill_value = 10**9
M_2017, meta_2017 = create_folium_map_clusters(OTHER_YEARS_ACCIDENTS['2017'],
                                               max_distance=MAX_DISTANCE,
                                               min_cluster_size=MIN_CLUSTER_SIZE,
                                               radius = 10)
M_2017

<h4 style="color:#ff9900">ACCIDENT CLUSTERS IN 2016</h4>

In [ ]:
M_2016, meta_2016 = create_folium_map_clusters(OTHER_YEARS_ACCIDENTS['2016'],
                                               max_distance=MAX_DISTANCE, 
                                               min_cluster_size=MIN_CLUSTER_SIZE,
                                               radius = 10)
M_2016

<h4 style="color:#ff9900">ACCIDENT CLUSTERS IN 2015</h4>

In [ ]:
M_2015, meta_2015 = create_folium_map_clusters(OTHER_YEARS_ACCIDENTS['2015'],
                                    max_distance=MAX_DISTANCE,
                                    min_cluster_size=MIN_CLUSTER_SIZE,
                                    radius = 10)
M_2015

<h3 style="color:green">Overview of accidents hotspots from past 5 years</h3>
<p>By looking at single years, it is hard to spot a trend, therefore we decided to compute centroids of clusters for each year. These centroids are then going to be plotted on one map disnguished by color representing the given year.</p>
<h4 style="color:#ff9900">GET RELEVANT DATA</h4>
<ul><li><b>META INFORMATION:</b> This information includes latitude, longitude and cluster label of the given point.</li>
<li><b>SELECTION OF DATA:</b> We take meta data for each year and for each cluster within the year, we select one centroid, therefore variable relevant data includes all 5 years and corresponding centroids for the given year.</li>
</ul>

In [ ]:
# First, merge all information into one dictionary first
meta_information = {'2019': meta_2019,
                    '2018': meta_2018,
                    '2017': meta_2017,
                    '2016': meta_2016,
                    '2015': meta_2015
                   }

five_years_summary = {
    year: map_points_to_cluster_centers(meta_info_year)
    for year, meta_info_year in meta_information.items()
}

# Parse the data into a list
years = []
latitude = []
longitude = []
for year, info in five_years_summary.items():
    for lat, lon in info.values():
        years.append(year)
        latitude.append(lat)
        longitude.append(lon)

# Second, create a structured numpy array with following structure
# - Year
# - Latitude
# - Longitude
vals = [(year, lat, lon) for year, lat, lon
        in zip(years,
               latitude,
               longitude)]

relevant_data = np.array(vals,
                         dtype=[('Year', int),
                                ('Latitude', float),
                                ('Longitude', float)])
relevant_data = np.ma.asarray(relevant_data)

<h4 style="color:#ff9900">ALL CENTROIDS PLOTTED ON ONE MAP</h4>
<ul>
<li><b>NOTE:</b> At this step, we take all centroids for the given year and plot them onto the map. Thus, the map below represents hotspots of accidents for the past 5 years in Manchester.</li>
</ul>

In [ ]:
color_map = {
        2019: '#170536',
        2018: '#e38c56',
        2017: '#ff0a0a',
        2016: '#039491',
        2015: '#940322',
}
M_All = create_folium_map(relevant_data, color_map=color_map, column_to_highlight='Year', radius=7.5)
add_legend(M_All, color_map)

<h4 style="color:#ff9900">CLUSTERS FROM CENTROIDS</h4>
<ul>
<li><b>NOTE:</b> Previous map shows a nice overview of hotspots, but are there any patterns over the years that keep repeating? We try to answer this question by clustering hotspots from all years.</li>
<li><b>NUMBER_OF_YEARS_PATTERN:</b> This is a very important hyper parameter, whose range for this case is from 2 to 5. The more longterm pattern you want to see, the greater this parameter must be indeed.</li>
</ul>

In [ ]:
# Cluster all the points from previous map
NUMBER_OF_YEARS_PATTERN = 4  # Set this parameter to see a pattern over the specified period of years

In [ ]:
_, meta_ALL = create_folium_map_clusters(relevant_data,
                                         max_distance=MAX_DISTANCE,
                                         min_cluster_size=NUMBER_OF_YEARS_PATTERN,
                                         radius=10)

# Map the clusters to the corresponding year: from meta_all you know the location which can be
# then used as a mask to find the year in relevant data array
location_year_centroids = map_location_to_year(meta_ALL, relevant_data)
color_map = {
        2019: '#170536',
        2018: '#f06800',
        2017: '#ff0a0a',
        2016: '#039491',
        2015: '#940322',
}
M_All_clustered = create_folium_map(location_year_centroids,
                          color_map=color_map,
                          column_to_highlight='Year',
                          radius=10)
add_legend(M_All_clustered, color_map)

<h3 style="color:green">Summarize the identified clusters</h3>
<p>This section tries to summarize selected hotspots.</p>
<h4 style="color:#ff9900">PUT TOGETHER A SUMMARY OF SELECTED HOTSPOTS</h4>
<ul>
<li><b>NOTE:</b> We first create a dictionary where keys are simply the clusters labels, and value is another dictionary which holds a year as key and values then represent detailed information about the accident from the given year. In other words, for each hotspot a summary is created and stored within variable hotspots_accidents_summary.</li>
</ul>

In [ ]:
hotspots_accidents_summary = dict()
for row in location_year_centroids:
    
    # Get year
    year = str(row['Year'])
    
    # Get relavant accidents based on the location from the centroid
    relevant_accidents = map_centroid_to_locations(row[['Latitude', 'Longitude']],
                             year,
                             ALL_YEARS_ACCIDENTS,
                             max_perimeter=MAX_DISTANCE)
    
    cluster_label = row['Label']
    if cluster_label in hotspots_accidents_summary:
        hotspots_accidents_summary[cluster_label][year] = relevant_accidents
    else:
        hotspots_accidents_summary[cluster_label] = {year: relevant_accidents}

<h4 style="color:#ff9900">TOTAL NUMBER OF ACCIDENTS PER CLUSTER</h4>
<ul>
<li><b>NOTE:</b> Our goal here is to identify where the biggest impact can be made by summing number of accidents related to each cluster.</li>
</ul>

In [ ]:
total_n_acc_cluster = dict()

for cluster_id, cluster_info in hotspots_accidents_summary.items():
    total = 0
    for year, accidents in cluster_info.items():
        total += accidents.shape[0]
    total_n_acc_cluster[cluster_id] = total
        
sorted_dic = {k: v for k, v in sorted(total_n_acc_cluster.items(), key=lambda item: item[1])}
top_n = list(sorted_dic.keys())[-3:]
print(f"The largest number of accidents happened at these clusters: {top_n}")
print(sorted_dic)

<h4 style="color:#ff9900">IDENTIFY THE TOP CLUSTERS ON THE MAP</h4>

In [ ]:
# HARD CODED DATA
map_view_dic = {
    6: 'https://www.google.com/maps/@53.4564752,-2.1997858,3a,75y,312.56h,84.6t/data=!3m7!1e1!3m5!1ssvkH8khlpyWpLfzgne_MAg!2e0!6s%2F%2Fgeo0.ggpht.com%2Fcbk%3Fpanoid%3DsvkH8khlpyWpLfzgne_MAg%26output%3Dthumbnail%26cb_client%3Dmaps_sv.tactile.gps%26thumb%3D2%26w%3D203%26h%3D100%26yaw%3D345.15714%26pitch%3D0%26thumbfov%3D100!7i16384!8i8192',
    4: 'https://www.google.com/maps/@53.4562744,-2.2257357,3a,75y,25.15h,89.12t/data=!3m6!1e1!3m4!1s5W0MsAbTfwMruiIMDaXPVQ!2e0!7i16384!8i8192',
    1: 'https://www.google.com/maps/@53.470424,-2.2229411,3a,75y,159.88h,89.58t/data=!3m6!1e1!3m4!1skeRSOklQINAEuHN-XKBQtw!2e0!7i16384!8i8192'
}


for cluster_id in top_n:
    
    # Get location of the given cluster
    mask_cluster_id = meta_ALL['Cluster'] == cluster_id
    
    # Compute the center of the given cluster - use simply a mean
    location = meta_ALL['Latitude'][mask_cluster_id].mean(), meta_ALL['Longitude'][mask_cluster_id].mean()
    
    const = 0.0005
    # Define upper bound
    upper = location[0] + const, location[1] + const
    # Define lower bound
    lower = location[0] - const, location[1] - const
    
    
    folium.Rectangle(bounds=[lower, upper], popup=map_view_dic[cluster_id], color='#ffee00', fill=True, fill_color='#ffee00', fill_opacity=0.2).add_to(M_All_clustered)
M_All_clustered